In [2]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct


import pandas as pd
import openai
import json
import tiktoken

## Load Amazon dataset(Items)

In [17]:
df_items = pd.read_json("../data/meta_Electronics_1000.jsonl", lines=True)
df_items.head(1)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,available_date
0,All Electronics,"Adjustable Metal Stand for E cho Show 15, Tilt...",4.5,111,[The VMEI adjustable metal bracket is made of ...,[],36.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Small footprint, minimalist design...",VMEI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'VMEI', 'Color': 'Black', 'Room Type...",B0B13T2GY5,NaN,2022-05-11


In [18]:
df_items_sample = df_items.sample(n=50, random_state=25)

len(df_items_sample)

50

### Preproc

In [19]:
def preprocess_data(row):
    return f"{row['title']} {' '.join(row['features'])}"

def extract_first_large_image(row):
    return row["images"][0].get("large", '')

df_items["preprocessed_data"] = df_items.apply(preprocess_data, axis=1)
df_items["first_large_image"] = df_items.apply(extract_first_large_image, axis=1)

df_items.head(1)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,available_date,preprocessed_data,first_large_image
0,All Electronics,"Adjustable Metal Stand for E cho Show 15, Tilt...",4.5,111,[The VMEI adjustable metal bracket is made of ...,[],36.99,[{'thumb': 'https://m.media-amazon.com/images/...,"[{'title': 'Small footprint, minimalist design...",VMEI,"[Electronics, Computers & Accessories, Compute...","{'Brand': 'VMEI', 'Color': 'Black', 'Room Type...",B0B13T2GY5,NaN,2022-05-11,"Adjustable Metal Stand for E cho Show 15, Tilt...",https://m.media-amazon.com/images/I/415fkdvFA9...
